# Writing Control to new collection

In [8]:
from config import db2
from json import loads

In [9]:
def clean(obj):
    """
    removes the keys with $ and adds the predict key
    """
    obj["_id"] = obj["_id"]["$oid"]
    del obj["timestamp"]
    return obj


def data_iter():
    with open("/Users/JasonLiu/Downloads/control-dump.json") as f:
        for line in f:
            obj = clean(loads(line))
            yield obj
            
def grouped(iterable, n):
    """
    Make an iterator of elements partitioned by the n
    :param iterable:
    :param n:
    """
    batch_window = [None for _ in range(n)]
    cur_size = 0
    for item in iterable:
        batch_window[cur_size] = item
        cur_size += 1
        if cur_size >= n:
            batched = batch_window[:]
            batch_window = [None for _ in range(n)]
            cur_size = 0
            yield batched

In [10]:
from itpy import Itpy

In [12]:
%%time
x = Itpy(data_iter()).filter(lambda _: _["random_number"] < 0.01)._

CPU times: user 1min 18s, sys: 1min 28s, total: 2min 46s
Wall time: 3min 12s


In [18]:
for i in grouped(x[108000:], 5000):
    db2.insert_many(i)

In [19]:
db2.count()

183000